## VacationPy

### Obervations





In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
# Export csv
cities_df = pd.read_csv("../output_data/city_data.csv")

# drop NA values
# cities_df = cities_df.dropna()
cities_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# configure gmaps
gmaps.configure(api_key = g_key)

In [ ]:
# Plot Heatmap

# Set up variables
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)
fig = gmaps.figure(center=(16.522731, -5.042275), zoom_level=2)

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display the map
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values

In [ ]:
#reduce cities to ideal conditions 
preferred_cities_df = cities_df.loc[(cities_df["Humidity"]<= 80) &(cities_df["Wind Speed"]<= 10) & (cities_df["Cloudiness"] == 0) & 
                                (cities_df["Max Temp"]>= 70) & (cities_df["Max Temp"]<= 80)].dropna()

preferred_cities_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Create a hotel_df
hotel_df = preferred_cities_df[["City","Country", "Lat", "Lng"]].copy()

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

In [ ]:
# Set up search criteria

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

In [ ]:
# Iterate through dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Print end of search once searching is completed
print("-------End of Search-------")

In [ ]:
# Print the first hotel info
first_hotel = hotel_df[["City","Country", "Hotel Name"]].copy()
first_hotel.head(1)

In [ ]:
# Print updated hotel dataframe
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add a marker layer on top of the heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer
fig.add_layer(marker_layer)

# Display figure
fig